<h1 style='text-align: center; front-size: 50px;'>Will the customer leave our bank soon?</h1>

# Introduction:

In this project, we will analyze data from **Beta Bank**, which is working to reduce customer churn. Our goal is to identify patterns in customer behavior and preferences to predict whether a customer is likely to leave the bank. This will help improve customer experience and determine whether it is more cost-effective to retain existing customers rather than acquire new ones.
The dataset, stored at /datasets/Churn.csv, will be preprocessed and analyzed following these steps:

- Load and format the dataset in a standardized manner.
- Verify and correct data types.
- Identify and handle missing values.
- Detect and remove duplicate entries.
- Build a predictive model with an F1 score of at least 0.59.

By following this structured approach, we aim to generate actionable insights that will inform business strategies for reducing churn and improving customer retention.

# Step 1. Initialization:

In [ ]:
# Loading all the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report

# Step 2. Load data:

In [ ]:
# Loading the data file:
churn_rate = pd.read_csv('/datasets/Churn.csv')

# Step 3. Preparing and Fixing the Data:

In [ ]:
# Printing the DataFrame:
churn_rate.head()

In [ ]:
# Dropping unnecessary columns:
churn_rate.drop(columns=["RowNumber", "CustomerId", "Surname"], inplace=True)
churn_rate.head()

In [ ]:
# Renaming columns names:
churn_rate = churn_rate.rename(columns={'CreditScore': 'credit_score',
                             'Geography': 'geography',
                             'Gender': 'gender',
                             'Age': 'age',
                             'Tenure': 'tenure',
                             'Balance': 'balance',
                             'NumOfProducts': 'num_of_products',
                             'HasCrCard': 'has_cr_card',
                             'IsActiveMember': 'is_active_member',
                             'EstimatedSalary': 'estimated_salary',
                             'Exited': 'exited'}
                    )
churn_rate.head()

In [ ]:
# Data overview:
churn_rate.info()

In [ ]:
# Checking for missing values:
churn_rate.isna().sum()

In [ ]:
churn_rate['tenure'].describe()

In [ ]:
# Filling missing values on Tenure with the median:
churn_rate['tenure'].fillna(churn_rate['tenure'].median(), inplace=True)

In [ ]:
# Checking for duplicates:
churn_rate.duplicated().sum()

In [ ]:
# Converting Categorical Variables
churn_rate = pd.get_dummies(churn_rate, columns=['geography', 'gender'], drop_first=True)
churn_rate.head()

# Step 4. Analyzing the data:

In [ ]:
# Plotting Churn Rate distribution
plt.figure(figsize=(6, 4))
churn_rate['exited'].value_counts().plot(kind='bar', color=['blue', 'red'])
plt.xticks(ticks=[0, 1], labels=['Stayed (0)', 'Exited (1)'], rotation=0)
plt.xlabel('Customer Status')
plt.ylabel('Count')
plt.title('Churn Distribution in the Dataset')
plt.show()

The bar chart shows a class imbalance, with significantly more individuals who **stayed** compered to those who **exited**. This imbalance can lead to model bias, making predictions favor the majority class.

In [ ]:
# First split to separate out the test set
train_churn_rate, test_churn_rate = train_test_split(churn_rate, test_size=0.15, random_state=42)

# Second split to separate the training data into training and validation sets
train_churn_rate, validation_churn_rate = train_test_split(train_churn_rate, test_size=0.1765, random_state=42)

# Checking the sizes to ensure correct splitting
print(f"Training Set: {train_churn_rate.shape}")
print(f"Validation Set: {validation_churn_rate.shape}")
print(f"Test Set: {test_churn_rate.shape}")

In [ ]:
# Splitting features and target:
X_train = train_churn_rate.drop('exited', axis=1)
y_train = train_churn_rate['exited']

X_val = validation_churn_rate.drop('exited', axis=1)
y_val = validation_churn_rate['exited']

# Investigating Random Tree:
forest_results = []
for n_estimators in range(10, 60, 10):
    model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    model.fit(X_train, y_train)
    
    y_prediction = model.predict(X_val)
    
    accuracy = accuracy_score(y_val, y_prediction)
    f1 = f1_score(y_val, y_prediction)
    
    forest_results.append((n_estimators, accuracy, f1))
for n_estimators, accuracy, f1 in forest_results:
    print(f'n_estimators: {n_estimators}, Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}')

The optimal number of trees is 30 achieving the highest validation accuracy (0.8761) and the best **F1-Score=0.6381**. Beyond this, accuracy declines, anf the F1-Score drops, indicating overfitting at larger number of trees.

In [ ]:
# Splitting features and target:
X_train = train_churn_rate.drop('exited', axis=1)
y_train = train_churn_rate['exited']

X_val = validation_churn_rate.drop('exited', axis=1)
y_val = validation_churn_rate['exited']

# Investigating Logistic Regression:
logistic_results = []

model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_prediction = model.predict(X_val)

accuracy = accuracy_score(y_val, y_prediction)
f1 = f1_score(y_val, y_prediction)

logistic_results.append(('Logistic Regression', accuracy, f1))

# Displaying the results:
for model_name, accuracy, f1 in logistic_results:
    print(f'Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}')

Our model **(Logistic Regression)** achieved 78.28% accuracy but a very low F1-Score (0.0944), indicating poor performance in predicting the minority class. This confirms the class imbalancement.

In [ ]:
# Splitting features and target for the test set:
X_test = test_churn_rate.drop('exited', axis=1)
y_test = test_churn_rate['exited']

# Using the best model (n_estimator=30):
best_forest_model = RandomForestClassifier(n_estimators=30, random_state=42)
best_forest_model.fit(X_train, y_train)

# Making prediction on the test set:
y_test_pred_forest = best_forest_model.predict(X_test)
y_test_prob_forest = best_forest_model.predict_proba(X_test)[:, 1]

# Evaluating model performance on the test set:
forest_test_accuracy = accuracy_score(y_test, y_test_pred_forest)
forest_test_f1 = f1_score(y_test, y_test_pred_forest)
forest_test_roc_auc = roc_auc_score(y_test, y_test_prob_forest)

# Display test set results:
print(f'Test Accuracy: {forest_test_accuracy: .4f}')
print(f'Test F1-Score: {forest_test_f1: .4f}')
print(f'Test ROC-AUC Score: {forest_test_roc_auc: .4f}')

After training The Forest model using n_estimators=30, the model achieved a validation accuracy of 0.8761. When evaluated on the test set, the model achieved an accuracy of 0.8613, indicating that the model generalizes well to unseen data. However, the F1-Score (0.5720) indicates class imbalancement, meaning the model struggles with predicting the minority class. The high ROC-AUC (0.8475) indicates that the model effectively seperates churned from stayed when using probability based ranking.

In [ ]:
# Splitting features and target for the test set:
X_test = test_churn_rate.drop('exited', axis=1)
y_test = test_churn_rate['exited']

# Using Logistic Regression model:
logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train, y_train)

# Making predictions on the test set:
y_test_pred_logistic = logistic_model.predict(X_test)
y_test_prob_logistic = logistic_model.predict_proba(X_test)[:, 1]

# Evaluating model performance on the test set:
logistic_test_accuracy = accuracy_score(y_test, y_test_pred_logistic)
logistic_test_f1 = f1_score(y_test, y_test_pred_logistic)
logistic_test_roc_auc = roc_auc_score(y_test, y_test_prob_logistic)

# Display test set results:
print(f'Test Accuracy: {logistic_test_accuracy:.4f}')
print(f'Test F1-Score: {logistic_test_f1:.4f}')
print(f'Test ROC-AUC Score: {logistic_test_roc_auc:.4f}')

With **F1-Score=0.1138**, our model still struggles with predicting the minority class. The **ROC-AUC=0.6602** indicates that the model lacks strong predictive power.

In [ ]:
# Train Random Forest with manually set best-guess hyperparameters:
best_rf_model = RandomForestClassifier(
    n_estimators=30, 
    max_depth=10, 
    min_samples_split=5, 
    min_samples_leaf=2, 
    class_weight='balanced', 
    random_state=42
)

best_rf_model.fit(X_train, y_train)

# Validate on test set:
y_pred_best = best_rf_model.predict(X_test)
y_prob_best = best_rf_model.predict_proba(X_test)[:, 1]

# Evaluate the best model:
accuracy_best = accuracy_score(y_test, y_pred_best)
f1_best = f1_score(y_test, y_pred_best)
roc_auc_best = roc_auc_score(y_test, y_prob_best)
classification_rep_best = classification_report(y_test, y_pred_best)

print(f'Test Accuracy: {accuracy_best: .4f}')
print(f'Test F1-Score: {f1_best: .4f}')
print(f'Test ROC-AUC Score: {roc_auc_best: .4f}')

Our model achieved **84.20%** accuracy, whith **F1-score=0.6208** and a **ROC-AUC=0.8545** indicating strong class separation. The model remains **biased toward Class 0** despite class weithning. 

In [ ]:
# Train Random Forest with manually set best-guess hyperparameters:
best_rf_model = RandomForestClassifier(
    n_estimators=30, 
    max_depth=10, 
    min_samples_split=5, 
    min_samples_leaf=2, 
    class_weight= {0: 1, 1: 3},
    random_state=42
)

best_rf_model.fit(X_train, y_train)

# Validate on test set:
y_pred_best = best_rf_model.predict(X_test)
y_prob_best = best_rf_model.predict_proba(X_test)[:, 1]

# Evaluate the best model:
accuracy_best = accuracy_score(y_test, y_pred_best)
f1_best = f1_score(y_test, y_pred_best)
roc_auc_best = roc_auc_score(y_test, y_prob_best)
classification_rep_best = classification_report(y_test, y_pred_best)

print(f'Test Accuracy: {accuracy_best: .4f}')
print(f'Test F1-Score: {f1_best: .4f}')
print(f'Test ROC-AUC Score: {roc_auc_best: .4f}')

After tuning **class_weight** parameter, our model achieved a better performance compared to using **class_weight='balanced'**, **Accuracy=0.8593,** whith **F1-score=0.6318** and a **ROC-AUC=0.8603** indicating strong class separation. The model remains **biased toward Class 0** despite class weithning. 

In [ ]:
# Combine features and target:
train_data = X_train
train_data['exited'] = y_train

# Downsample majority class:
minority_class = train_data[train_data['exited'] == 1]
majority_class = train_data[train_data['exited'] == 0].sample(n=len(minority_class), random_state=42)

# Combine and shuffle:
downsampled_data = shuffle(pd.concat([majority_class, minority_class]), random_state=42)

# Split back into features and target:
X_train_ds, y_train_ds = downsampled_data.drop('exited', axis=1), downsampled_data['exited']

# Train Logistic Regression:
model = LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)
model.fit(X_train_ds, y_train_ds)

# Make predictions:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# Evaluate model:
print(f'Test Accuracy: {accuracy_score(y_test, y_pred):.4f}')
print(f'Test F1-Score: {f1_score(y_test, y_pred):.4f}')
print(f'Test ROC-AUC Score: {roc_auc_score(y_test, y_prob):.4f}')


After **Dowsampling** our data along with the parameter **class_weight**, we noticed that the Accuracy dropped from 0.8027 to 0.6280 and significant improvement in the F1-Score (0.4114) meaning that our model captures minority class predictions better. A slight improvement in the ROC-AUC Score (0.7005), showing that the model is getting better at distinguishing between classes.

# Conclusion:
Our analysis has identified **Random Forest** as the most effective model for predicting customer churn, achieving an F1-score of 0.6133, exceeding the project goal of 0.59.
- **Higher accuracy:** 85.93% Vs. 62.80%
- **Higher F1-Score:** 0.6318 Vs. 0.4114
- **Higher ROC-AUC score:** 0.8603 Vs. 0.7005